In [ ]:
import numpy as np # linear algebre
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

We'll use librosa to read audio and perform some analysis

In [ ]:
import librosa
import soundfile as sf
import librosa.display as lrd

import os

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

Let's read both tables to identify true and false positives on the spectrum

In [ ]:
tpdf = pd.read_csv('/kaggle/input/rfcx-species-audio-detection/train_tp.csv')
fpdf = pd.read_csv('/kaggle/input/rfcx-species-audio-detection/train_fp.csv')

In [ ]:
tpdf

In [ ]:
fpdf

## Number of samples per call type and species

In [ ]:
tpdf['duration'] = tpdf.t_max-tpdf.t_min
tpdf['bandwidth'] = tpdf.f_max-tpdf.f_min

tpdf.pivot_table(index='species_id',columns='songtype_id',
                 values='duration',aggfunc='count').fillna(0)

## Metadata per species

Notice how the duration and frequency bands are highly characteristic of each species. 

The trouble will be to establish the event boundaries in new sound files

### Call duration


In [ ]:
fig,ax = plt.subplots(1,figsize=(10,4))
sns.boxplot(data=tpdf,y='duration',x='species_id',hue='songtype_id')

### Lower frequency of call band

In [ ]:
fig,ax = plt.subplots(1,figsize=(10,4))
sns.boxplot(data=tpdf,y='f_min',x='species_id',hue='songtype_id')

### Uper frequency of call band

In [ ]:
fig,ax = plt.subplots(1,figsize=(10,4))
sns.boxplot(data=tpdf,y='f_max',x='species_id',hue='songtype_id')

### Bandwidth

In [ ]:
fig,ax = plt.subplots(1,figsize=(10,4))
sns.boxplot(data=tpdf,y='bandwidth',x='species_id',hue='songtype_id')

## Investigate Species 0

In [ ]:
tpdf

In [ ]:
s0 = tpdf[tpdf['species_id'] == 0]
s0

In [ ]:
r0 = s0[s0['recording_id'] == '00d442df7']
r0

In [ ]:
t_min = r0['t_min'].values[0]
t_max = r0['t_max'].values[0]

In [ ]:
!ls /kaggle/input/rfcx-species-audio-detection/train

In [ ]:
import IPython
IPython.display.Audio("/kaggle/input/rfcx-species-audio-detection/train/00d442df7.flac")

In [ ]:
data, samplerate = librosa.load('/kaggle/input/rfcx-species-audio-detection/train/00d442df7.flac', mono=True)
IPython.display.Audio(data, rate=samplerate)

In [ ]:
samplerate

In [ ]:
data.shape

In [ ]:
data.shape[0] / samplerate

In [ ]:
start_point = int(samplerate * t_min)
start_point

In [ ]:
end_point = int(samplerate * t_max)
end_point

In [ ]:
r0_cut = data[start_point:end_point]

IPython.display.Audio(r0_cut, rate=samplerate)

In [ ]:
r0_cut.shape

In [ ]:
S0_cut = np.abs(librosa.stft(r0_cut))

In [ ]:
S0_cut.shape

In [ ]:
fig, ax = plt.subplots()
img = librosa.display.specshow(librosa.amplitude_to_db(S0_cut, ref=np.max), y_axis='log', x_axis='time', ax=ax)
ax.set_title('Power spectrogram')
fig.colorbar(img, ax=ax, format="%+2.0f dB")

In [ ]:
IS0_cut = librosa.istft(S0_cut)

IPython.display.Audio(IS0_cut, rate=samplerate)

In [ ]:
tpdf['species_id'].unique()

In [ ]:
# 0-23 different species
# species_unique_list = sorted(tpdf['species_id'].unique())
species_unique_list = [0]
species_0 = {}
for species in species_unique_list:
    species_df = tpdf[tpdf['species_id'] == species]
    
    for i in species_df.iterrows():
        record_id = i[1][0]
        t_min = i[1][3]
        t_max = i[1][5]
        
        data, samplerate = sf.read('/kaggle/input/rfcx-species-audio-detection/train/'+record_id+'.flac')
        
        start_point = int(samplerate * t_min)
        end_point = int(samplerate * t_max)
    
        cut = data[start_point:end_point]
        
        stft_cut = np.abs(librosa.stft(cut))
        
        # Currently we get only same sized sounds
        if stft_cut.shape[1] != 75:
            continue
        
        species_0[record_id] = stft_cut

species_0

## GAN Tensorflow2

In [ ]:
import tensorflow as tf
import random
from tensorflow import keras
import glob
import matplotlib.pyplot as plt
import time
from tensorflow.keras import layers
import os

In [ ]:
# We have 40 record for species 0. 10 of them are longer so we dont use them.
data = np.array(list(species_0.values()))
data.shape

In [ ]:
# 1 is for channel
data = data.reshape(40, 1025, 75, 1)
data.shape

In [ ]:
BATCH_SIZE = 8
BUFFER_SIZE = 6
train_dataset = tf.data.Dataset.from_tensor_slices(data).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
train_dataset

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(41*5*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((41, 5, 256)))
    assert model.output_shape == (None, 41, 5, 256) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 41, 5, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(5, 3), padding='same', use_bias=False))
    assert model.output_shape == (None, 205, 15, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(5, 5), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 1025, 75, 1)

    return model

In [ ]:
# Generate random stft
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_stft = generator(noise, training=False)

generated_stft_resized = generated_stft.numpy().reshape(1025, 75)

fig, ax = plt.subplots()
img = librosa.display.specshow(librosa.amplitude_to_db(generated_stft_resized, ref=np.max), y_axis='log', x_axis='time', ax=ax)
ax.set_title('Power spectrogram')
fig.colorbar(img, ax=ax, format="%+2.0f dB")

In [ ]:
generated_sound = librosa.istft(generated_stft_resized)

IPython.display.Audio(generated_sound, rate=samplerate)

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(5, 5), padding='same', input_shape=[1025, 75, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(5, 3), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
discriminator = make_discriminator_model()
decision = discriminator(generated_stft)
print(decision)

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
EPOCHS = 90000
noise_dim = 100
num_examples_to_generate = 1

# We will reuse this seed overtime (so it's easier)
# to visualize progress
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(records):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_records = generator(noise, training=True)

        real_output = discriminator(records, training=True)
        fake_output = discriminator(generated_records, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        
        for image_batch in dataset:
            train_step(image_batch)
            
        # Save the model every 500 epochs
        if (epoch + 1) % 250 == 0:
            # Produce records as we go
            generate_and_save_records(generator, epoch + 1, seed)
            
        IPython.display.clear_output(wait=True)
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    # Generate after the final epoch
    IPython.display.clear_output(wait=True)
    generate_and_save_records(generator, epochs, seed)

In [ ]:
def generate_and_save_records(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    generated_stfts = predictions.numpy().reshape(num_examples_to_generate, 1025, 75)
    for i in range(generated_stfts.shape[0]):
        generated_stft = generated_stfts[i]
        generated_sound = librosa.istft(generated_stft)
        sf.write('record_at_epoch_{}_{}th.ogg'.format(epoch, i), generated_sound, samplerate)

In [ ]:
train(train_dataset, EPOCHS)

In [ ]:
data, samplerate = librosa.load('record_at_epoch_50000_0th.ogg', mono=True)
IPython.display.Audio(data, rate=samplerate)

## Classification

We will use 25(5val+20train) generated and 25(5val+20train) real records for species_0

40(10val+30train) real records for species_1

40(10val+30train) real records for species_2

Total train size 100 and validation size will be 30

#### Species 0

In [ ]:
# We will use 25(5val+20train) generated and 25(5val+20train) real records for species_0
# 40(10val+30train) real records for species_1
# 40(10val+30train) real records for species_2

# Total train size 100 and validation size will be 30
species_0_list = np.array(list(species_0.values()))
species_0_real_train = species_0_list[:20]
species_0_real_train.shape

In [ ]:
species_0_real_validation = species_0_list[20:25]
species_0_real_validation.shape

In [ ]:
species_0_generated_train = []
for i in range(20):
    noise = tf.random.normal([1, 100])
    generated_record = generator(noise, training=False)

    generated_record = generated_record.numpy().reshape(-1, 64000)
    species_0_generated_train.append(generated_record)

    
species_0_generated_train = np.array(species_0_generated_train).squeeze()
species_0_generated_train.shape

In [ ]:
species_0_generated_validation = []
for i in range(5):
    noise = tf.random.normal([1, 100])
    generated_record = generator(noise, training=False)

    generated_record = generated_record.numpy().reshape(-1, 64000)
    species_0_generated_validation.append(generated_record)
    
species_0_generated_validation = np.array(species_0_generated_validation).squeeze()
species_0_generated_validation.shape

In [ ]:
# 40 species_0 record will use as train(20 generated and 20 real)
train_species_0 = np.concatenate((species_0_generated_train, species_0_real_train))
train_species_0.shape

In [ ]:
# Create labels species_0 is 0 and species_1 is 1 and species_3 is 2
train_species_0_label = np.zeros(len(train_species_0))
train_species_0_label.shape

In [ ]:
# 20 species_0 record will use as train(5 generated and 5 real)
validation_species_0 = np.concatenate((species_0_generated_validation, species_0_real_validation))
validation_species_0.shape

In [ ]:
validation_species_0_label = np.zeros(len(validation_species_0))
validation_species_0_label.shape

#### Species 1

In [ ]:
# 0-23 different species
# species_unique_list = sorted(tpdf['species_id'].unique())
species_unique_list = [1]
species_1 = {}
for species in species_unique_list:
    species_df = tpdf[tpdf['species_id'] == species]
    
    for i in species_df.iterrows():
        record_id = i[1][0]
        t_min = i[1][3]
        t_max = i[1][5]
        
        data, samplerate = sf.read('/kaggle/input/rfcx-species-audio-detection/train/'+record_id+'.flac')
        
        start_point = int(samplerate * t_min)
        end_point = int(samplerate * t_max)
    
        cut = data[start_point:end_point]
        
        species_1[record_id] = cut

species_1

In [ ]:
# Padding every record to 64000 samples

'''
max_len = -1
for record_data in species_0.values():
    length = len(record_data)
    if length > max_len:
        max_len = length
'''
max_len = 64000
'''
a = [1, 2, 3, 4, 5]
np.pad(a, (3), 'constant', constant_values=(6))

>>> array([6, 6, 6, 1, 2, 3, 4, 5, 6, 6, 6])
'''
for record_id in species_1.keys():
    if len(species_1[record_id]) < max_len:
        species_1[record_id] = np.pad(species_1[record_id], (0, (max_len - len(species_1[record_id]))), 'constant', constant_values=(0))

species_1

In [ ]:
IPython.display.Audio(species_1['0151b7d20'], rate=samplerate)

In [ ]:
species_1_list = np.array(list(species_1.values()))
train_species_1 = species_1_list[:30]
train_species_1.shape

In [ ]:
# Create labels species_0 is 0 and species_1 is 1 and species_3 is 2
train_species_1_label = np.ones(len(train_species_1))
train_species_1_label.shape

In [ ]:
validation_species_1 = species_1_list[30:40]
validation_species_1.shape

In [ ]:
# Create labels species_0 is 0 and species_1 is 1 and species_3 is 2
validation_species_1_label = np.ones(len(validation_species_1))
validation_species_1_label.shape

#### Species 3

In [ ]:
# 0-23 different species
# species_unique_list = sorted(tpdf['species_id'].unique())
species_unique_list = [3]
species_3 = {}
for species in species_unique_list:
    species_df = tpdf[tpdf['species_id'] == species]
    
    for i in species_df.iterrows():
        record_id = i[1][0]
        t_min = i[1][3]
        t_max = i[1][5]
        
        data, samplerate = sf.read('/kaggle/input/rfcx-species-audio-detection/train/'+record_id+'.flac')
        
        start_point = int(samplerate * t_min)
        end_point = int(samplerate * t_max)
    
        cut = data[start_point:end_point]
        
        species_3[record_id] = cut

species_3

In [ ]:
# Padding every record to 64000 samples

'''
max_len = -1
for record_data in species_0.values():
    length = len(record_data)
    if length > max_len:
        max_len = length
'''
max_len = 64000
'''
a = [1, 2, 3, 4, 5]
np.pad(a, (3), 'constant', constant_values=(6))

>>> array([6, 6, 6, 1, 2, 3, 4, 5, 6, 6, 6])
'''
for record_id in species_3.keys():
    if len(species_3[record_id]) < max_len:
        species_3[record_id] = np.pad(species_3[record_id], (0, (max_len - len(species_3[record_id]))), 'constant', constant_values=(0))

species_3

In [ ]:
species_3_list = np.array(list(species_3.values()))
train_species_3 = species_3_list[:30]
train_species_3.shape

In [ ]:
# Create labels species_0 is 0 and species_1 is 1 and species_3 is 2
train_species_3_label = np.ones(len(train_species_3)) * 2
train_species_3_label.shape

In [ ]:
validation_species_3 = species_3_list[30:40]
validation_species_3.shape

In [ ]:
# Create labels species_0 is 0 and species_1 is 1 and species_3 is 2
validation_species_3_label = np.ones(len(validation_species_3)) * 2
validation_species_3_label.shape

#### Concatenate data

In [ ]:
X_train = np.concatenate((train_species_0, train_species_1, train_species_3)).reshape(100, 64000, 1)
y_train = np.concatenate((train_species_0_label, train_species_1_label, train_species_3_label)).reshape(100, 1)
X_validation = np.concatenate((validation_species_0, validation_species_1, validation_species_3)).reshape(30, 64000, 1)
y_validation = np.concatenate((validation_species_0_label, validation_species_1_label, validation_species_3_label)).reshape(30, 1)

In [ ]:
print('X_train shape,', X_train.shape)
print('y_train shape,', y_train.shape)
print('X_validation shape,', X_validation.shape)
print('y_validation shape,', y_validation.shape)

In [ ]:
y_validation

#### Preprocess

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder()
y_validation = enc.fit_transform(y_validation).toarray()
y_validation

In [ ]:
enc = OneHotEncoder()
y_train = enc.fit_transform(y_train).toarray()

#### Create model

In [ ]:
def make_classification_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv1D(32, (25), strides=(4), padding='same',
                                     input_shape=[64000,1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv1D(32, (25), strides=(4), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv1D(64, (25), strides=(4), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv1D(32, (25), strides=(4), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv1D(32, (25), strides=(4), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))

    return model

In [ ]:
classification_model = make_classification_model()

In [ ]:
classification_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
classification_model.fit(X_train, y_train, validation_data=(X_validation, y_validation), epochs=150, batch_size=8)

#### Test model

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# We never use species_0 records from 25 to 50
species_0_test = species_0_list[25:50]
species_0_test.shape

In [ ]:
species_0_test = species_0_test.reshape(25, 64000, 1)
species_0_test.shape

In [ ]:
classification_model.predict_classes(species_0_test)

In [ ]:
accuracy_score(classification_model.predict_classes(species_0_test), np.zeros(len(species_0_test)))

In [ ]:
#We never use species_1 records from 40 to 50
species_1_test = species_1_list[40:50]
species_1_test.shape

In [ ]:
species_1_test = species_1_test.reshape(10, 64000, 1)
species_1_test.shape

In [ ]:
classification_model.predict_classes(species_1_test)

In [ ]:
accuracy_score(classification_model.predict_classes(species_1_test), np.ones(len(species_1_test)))

In [ ]:
#We never use species_3 records from 40 to 50
species_3_test = species_3_list[40:50]
species_3_test.shape

In [ ]:
species_3_test = species_3_test.reshape(10, 64000, 1)
species_3_test.shape

In [ ]:
classification_model.predict_classes(species_3_test)

In [ ]:
accuracy_score(classification_model.predict_classes(species_3_test), np.ones(len(species_3_test))*2 )

## Classification 2

We never use real records for species_0 this time

#### Species 0

In [ ]:
# We will use 40(10val+30train) generated records for species_0
# 40(10val+30train) real records for species_1
# 40(10val+30train) real records for species_2

# Total train size 90 and validation size will be 30

In [ ]:
species_0_generated = []
for i in range(40):
    noise = tf.random.normal([1, 100])
    generated_record = generator(noise, training=False)

    generated_record = generated_record.numpy().reshape(-1, 64000)
    species_0_generated.append(generated_record)

    
species_0_generated = np.array(species_0_generated).squeeze()
species_0_generated.shape

In [ ]:
train_species_0 = species_0_generated[:30]
train_species_0.shape

In [ ]:
validation_species_0 = species_0_generated[30:40]
validation_species_0.shape

In [ ]:
# Create labels species_0 is 0 and species_1 is 1 and species_3 is 2
train_species_0_label = np.zeros(len(train_species_3))
train_species_0_label.shape

In [ ]:
# Create labels species_0 is 0 and species_1 is 1 and species_3 is 2
validation_species_0_label = np.zeros(len(validation_species_3))
validation_species_0_label.shape

#### Concatenate

In [ ]:
X_train = np.concatenate((train_species_0, train_species_1, train_species_3)).reshape(90, 64000, 1)
y_train = np.concatenate((train_species_0_label, train_species_1_label, train_species_3_label)).reshape(90, 1)
X_validation = np.concatenate((validation_species_0, validation_species_1, validation_species_3)).reshape(30, 64000, 1)
y_validation = np.concatenate((validation_species_0_label, validation_species_1_label, validation_species_3_label)).reshape(30, 1)

In [ ]:
print('X_train shape,', X_train.shape)
print('y_train shape,', y_train.shape)
print('X_validation shape,', X_validation.shape)
print('y_validation shape,', y_validation.shape)

#### Preprocces

In [ ]:
enc = OneHotEncoder()
y_validation = enc.fit_transform(y_validation).toarray()
y_validation

In [ ]:
enc = OneHotEncoder()
y_train = enc.fit_transform(y_train).toarray()

#### Create second model

In [ ]:
classification_model_2 = make_classification_model()

In [ ]:
classification_model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
classification_model.fit(X_train, y_train, validation_data=(X_validation, y_validation), epochs=150, batch_size=8)

In [ ]:
# We never use real species_0 records
species_0_test = species_0_list[0:50]
species_0_test.shape

In [ ]:
species_0_test = species_0_test.reshape(50, 64000, 1)
species_0_test.shape

In [ ]:
classification_model_2.predict_classes(species_0_test)

In [ ]:
accuracy_score(classification_model_2.predict_classes(species_0_test), np.zeros(len(species_0_test)))

In [ ]:
# We never use last 10 species_1 records
species_1_test = species_1_list[40:50]
species_1_test.shape

In [ ]:
species_1_test = species_1_test.reshape(10, 64000, 1)
species_1_test.shape

In [ ]:
classification_model_2.predict_classes(species_1_test)

In [ ]:
accuracy_score(classification_model_2.predict_classes(species_1_test), np.ones(len(species_1_test)))

In [ ]:
# We never use last 10 species_3 records
species_3_test = species_3_list[40:50]
species_3_test.shape

In [ ]:
species_3_test = species_3_test.reshape(10, 64000, 1)
species_3_test.shape

In [ ]:
classification_model_2.predict_classes(species_3_test)

In [ ]:
accuracy_score(classification_model_2.predict_classes(species_3_test), np.ones(len(species_3_test))*2)